In [2]:
import numpy as np
import pandas as pd

# 쥬피터와 DataFrame의 출력을 소수점 이하 3자리로 제한
%precision 3
pd.set_option("display.precision", 3)

df = pd.read_csv("../data/ch2_scores_em.csv", index_col="student number")

en_scores = np.array(df["english"])[:10]
ma_scores = np.array(df["mathematics"])[:10]

scores_df = pd.DataFrame({"english": en_scores, "mathematics": ma_scores}, index=pd.Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], name="student"))
scores_df

,english,mathematics
student,,
A,42,65
B,69,80
C,56,63
D,41,63
E,57,76
F,48,60
G,65,81
H,49,66
I,65,78


# 3.1 두 데이터의 사이의 관계를 나타내는 지표
영어 점수가 높은 학생일수록 수학 점수도 높은 경향이 있다면, 영어 점수와 수학 점수는 **양의 상관**관계에 있다고 한다.
거꾸로 영어 점수가 높은 학생일수록 수학 점수가 낮은 경향이 있다면, 영어 점수와 수학 점수는 **음의 상관**관계에 있다고 한다.
그 어느쪽도 해당하지 않고 영어 점수가 수학 점수에 직접적으로 영향을 미치지 않을 때, **무상관**이라고 한다.

## 3.1.1 공분산
양의 상관관계에 있다고 생각되는 데이터의 관계성을 어떻게 수치화하면 좋을까?
**공분산**(covariance)이라는 지표를 사용한다.
공분산은 분산에 가까운 지표다.

공분산이 분산과 다른 점은, 가로축과 세로축의 데이터가 다르므로 편차들로 만든 도형이 직사각형이 될 분만 아니라 음의 면적도 얻을 수 있다는 점이다.
(분산은 가령 편차가 음의 값이어도 면적은 제곱값이므로 면적은 항상 양의 값이 된다.)

이철머 '부호를 붙인 면적'이라는 관점에서 면적이 양의 값ㅇ리 되는 것은 영어 점수와 수학 점수가 모두 평균보다 높거나 모두 평균 점수보다 낮은 경우이므로 점수의 상관관계를 잘 나타낸다고 볼 수 있다.

공분산이 양의 값이면 양의 값이 되는 데이터가 많다는 뜻이므로 양의 상관관계에 있고, 반대로 공분산이 음의 값이면 면적이 음의 값이 되는 데이터가 많다느 ㄴ뜻이므로 음으 상관관계에 있다고 할 수 있따. 그 어느쪽도 아니고 공분산이 0에 가까우면 무상관을 나타낸다.

In [8]:
summary_df = scores_df.copy()
summary_df["english_deviation"] = summary_df["english"] - summary_df["english"].mean()
summary_df["mathematics_deviation"] = summary_df["mathematics"] - summary_df["mathematics"].mean()
summary_df["product of deviations"] = summary_df["english_deviation"] * summary_df["mathematics_deviation"]
summary_df

,english,mathematics,english_deviation,mathematics_deviation,product of deviations
student,,,,,
A,42,65,-13.0,-6.4,83.2
B,69,80,14.0,8.6,120.4
C,56,63,1.0,-8.4,-8.4
D,41,63,-14.0,-8.4,117.6
E,57,76,2.0,4.6,9.2
F,48,60,-7.0,-11.4,79.8
G,65,81,10.0,9.6,96.0
H,49,66,-6.0,-5.4,32.4
I,65,78,10.0,6.6,66.0


In [10]:
summary_df["product of deviations"].mean()
# 62.800 -> 양수이므로 영어 점수와 수학 점수는 양의 상관관계에 잇다고 할 수 있다.

62.800

Numpy의 경우, 공분산은 cov 함수로 구할 수 있다. 다만, 반환값은 공분산값이 아닌 **공분산 행렬**또는 **분산공분산 행렬**이라고 부르는 행렬이다.

In [18]:
cov_mat = np.cov(en_scores, ma_scores, ddof=0)
cov_mat

array([[86.  , 62.8 ],
       [62.8 , 68.44]])

이 행렬의 첫번째 행과 첫 번째 열이 첫 번째 인수인 영어, 두 번째 행과 두번째 열이 두번째 인수인 수학에 각각 대응하고,
이것들의 교차하여 1행2열 성분과 2행 1열 성분이 영어와 수학의 공분산에 해당한다.
파이썬 인덱스는 0부터 시작하므로 결국 cov_mat의 [0, 1] 성분과 [1, 0] 성분이 공분산이 된다.

[0,0] -> 영어의 분산
[1,1] -> 수학의 분산

In [19]:
np.var(en_scores, ddof=0), np.var(ma_scores, ddof=0) # 분산과 일치한다.

(86.000, 68.440)

## 3.1.2 상관계수
앞에서 공분산을 계산하여 데이터의 상관관계를 표현했다.
시험 점수들 간에 공분산은 (점수 x 점수)라는 단위를 사용한다.
그러나 학생의 키와 시험 점수의 상관관계를 살펴본다면, (cm x 점수)라는 단위가 사용된다. 분산과 마찬가지로 이런 단위는 직감적으로 이해하기 어렵다.

따라서 단위에 의존하지 않는 상관을 나타내는 지표가 필요하다. 공분산은 각 데이터의 단위를 곱한 것이므로, 공분산을 각 데이터의 표준편차로 나누어 단위에 의존하지 않는 지표를 정의할 수 있다.

이와 같이 정의된 지표를 **상관 계수**(correlation coefficient)라고 한다.
상관계수는 반드시 -1과 1 사이의 값을 취하고, 데이터가 양의 상관관계에 놓여 있을수록 1에 가까워지고, 음의 상관관계에 놓여 있을수록 -1에 가까워진다. 무상관이면 0이 된다.
상관계수가 -1일때와 1일때에는 데이터가 완전히 직선상에 놓인다.

In [20]:
# 영어 점수와 수학 점수의 상관 계수를 구해보자
np.cov(en_scores, ma_scores, ddof=0)[0, 1] / (np.std(en_scores) * np.std(ma_scores))

0.819

In [21]:
# Numpy의 경우 상관계수는 corrcoef 함수로 계산할 수 있다. 다만, 반환값은 공분산의 경우와 마찬가지로 상관행렬이라고 부르는 행렬이다.
np.corrcoef(en_scores, ma_scores)

array([[1.   , 0.819],
       [0.819, 1.   ]])

In [22]:
# DataFrame의 경우, corr 메서드와 동일한 결과를 얻을 수 있다.
scores_df.corr()

,english,mathematics
english,1.000,0.819
mathematics,0.819,1.000
